In [95]:
# Для работы с таблицами импортируем pandas
import pandas as pd 

**Задание:**
Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA-тест, проверьте данные на нормальность и на наличие корреляции.

Для этого:
Объедините два массива в DataFrame. 
Обоснуйте выбор теста на корреляцию. 
Сделайте вывод по гипотезе.
Ответьте на вопрос: Какова разница в среднем размере мидии в зависимости от города-производителя
Обоснуйте выбор теста. 
Сделайте вывод по гипотезе.

**Работа с данными**

In [96]:
# Данные с которыми работаем
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]


In [97]:
# Преобразуем данные в DataFrame и объединяем их
Mussel_petersburg = pd.DataFrame(petersburg, columns = ['Mussels_Petersburg'])
Mussel_magadan = pd.DataFrame(magadan, columns = ['Mussels_Magadan'])
data = Mussel_petersburg.join(Mussel_magadan, how = 'outer')

In [98]:
# Смотрим как выглядит таблицы
print(data)
# Есть пропуск

   Mussels_Petersburg  Mussels_Magadan
0              0.0974           0.1033
1              0.1352           0.0915
2              0.0817           0.0781
3              0.1016           0.0685
4              0.0968           0.0677
5              0.1064           0.0697
6              0.1050           0.0764
7                 NaN           0.0689


In [99]:
# Заполним пропуск в столбце Mussels_Petersburg медианной
values = {'Mussels_Petersburg': data['Mussels_Petersburg'].median()}
data = data.fillna(values)

In [102]:
# Смотрю как выглядит таблицы
print(data)
# Согласно здравому смыслу выбросов не вижу

   Mussels_Petersburg  Mussels_Magadan
0              0.0974           0.1033
1              0.1352           0.0915
2              0.0817           0.0781
3              0.1016           0.0685
4              0.0968           0.0677
5              0.1064           0.0697
6              0.1050           0.0764
7              0.1016           0.0689


Тест на нормальность Д’Агостино

In [101]:
# Установливаем normaltest для определения нормального распределения
from scipy.stats import normaltest

# Установливаем гипотезы для нормальности
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально'

# Устанавливаем уровень значимости
alpha = 0.05

In [61]:
# Выполняю тест на нормальность
_, p = normaltest(data)

# Normaltest возвращает двустороннюю вероятность для проверки гипотезы
print('p=%.3f' % p[0])

# Интерпретация 
if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.035
Данные распределены нормально


**Корреляция**

Выбрала тест на корреляцию **Спирмена**, так как у признаков нет линейной связи. Я не могу сказать, что "С уменьшением/увеличением признака 1 уменьшается/увеличивается признак 2 в соответствии с некоторым уравнением прямой". 
Иногда можно применять ранговую корреляцию для числовых переменных если между ними существует нелинейная связь. 

In [109]:
# Импортируем spearmanr
from scipy.stats import spearmanr

# Установливаем гипотезы для корреляции
H0 = 'Отсутсвует зависимость между переменными.'
Ha = 'Есть зависимость между переменными.'

# Выполняем тест Спирмена
corr, p = spearmanr(data)
if p>alpha:
    print(f'{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}')
else:
    print(f'{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}')

0.712761707999163 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Отсутсвует зависимость между переменными.


Вывод: Согласно тесту Спирмена мы не можем отвергнуть нулевую гипотезу. Между переменными отсутствует зависимость. Следовательно, эти данные не имеют взаимосвязи и не влияют друг на друга.

**Какова разница в среднем размере мидии в зависимости от города-производителя**

Выбрала для сравнения тест T-тест, так как переменные у меня распределены нормально, тип зависимой переменной (размер мидии) является количественным. Так же у меня 2 группы сравнения и количество объектов меньше 30.

In [108]:
# Импортируем spearmanr
from scipy.stats import ttest_ind

# Установливаем гипотезы
H0 = 'Нет значимой разницы между средним размером раковины мидий в двух разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в двух разных местах.'

# Т-тест
def t_test(df):
    test_results = ttest_ind(df['Mussels_Petersburg'], df['Mussels_Magadan'], equal_var=True)

    p = round(test_results[1],3)

    if p>alpha:
        print(f'{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}')
    else:
        print(f'{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}')

t_test(data)

0.003 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в двух разных местах.


Вывод: Согласно Т-тесту мы отвергаем нулевую гипотезу. Между средним размером раковины мидий в двух разных местах есть значимая разница. Следовательно, место добыче мидии влияет на размер раковины. Значит данная информация может помочь определить место, где лучше ловить мидии, после дополнительного анализа по их общему количеству в том месте, а также их безопасности для пищи.

In [112]:
# Вычесляем разницу в среднем размере мидии в зависимости от города
differ_mussels = round((data['Mussels_Petersburg'].mean() - data['Mussels_Magadan'].mean()),3)
print(differ_mussels)

0.025


Какова разница в среднем размере мидии в зависимости от города-производителя?
Согласно тесту разница значима. В среднем размер мидии в Питере на 0.025 больше чем в Магадане.